In [1]:
%load_ext autoreload
%autoreload 2

import helpers as s

import OAI

In [2]:
import os, openai, datetime, hashlib, re
import time
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

import pandas as pd


from dotenv import load_dotenv
load_dotenv()

True

In [3]:
if os.getenv("OAI") is not None:
    openai.api_key = os.getenv("OAI")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OAI"))


OPENAI_API_KEY is ready


/home/kelu/projets/substack/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
signals = pd.read_parquet("data/signals.parquet.gzip")
metatags = pd.read_parquet("data/metatags.parquet.gzip")

In [5]:
df = pd.read_parquet('data/articles.parquet.gzip')
df.columns=  ["src","content","LEN"]
df = df[(df.LEN > 1500) & (df.LEN < 30000)].reset_index(drop=True)
titles = pd.read_parquet("data/titles.parquet.gzip")
df = df.merge(titles,on="src",how="left")
mt = pd.read_parquet("data/metatags.parquet.gzip")
df = df.merge(mt,on="src",how="left")
df.to_parquet("data/consolidated.parquet.gzip",compression="gzip")

In [6]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [7]:
articles = intersection(metatags.src.unique(), signals.src.unique() )
signals = signals[signals.src.isin(articles)]
metatags = metatags[metatags.src.isin(articles)]
len(articles)

1115

It takes 4 mins for 1000 articles. Approx.

In [8]:
base_path = "./DB/"

if not os.path.isfile(base_path+"chroma.sqlite3"):
    print("Start a new DB")
    vectordb = Chroma.from_documents(
        documents=[chunked_documents[0]],
        embedding=embeddings,
        persist_directory=base_path
    )
    vectordb.persist()
else:
    print("Continue on the DB")
    vectordb = Chroma(persist_directory=base_path,embedding_function=embeddings)
    print(len(vectordb.get()["ids"]),"elements already stored.")
    LSDOCS = vectordb.get()["documents"]

Continue on the DB
1410 elements already stored.


In [9]:
def getClosest(txt):
    B = vectordb.similarity_search(txt,6)
    L = []
    X = []
    for b in list(B):
        #print(b.metadata["src"],b.metadata["title"])
        L.append([b.metadata["src"],b.metadata["title"]])
        X.append(b.metadata["src"])
    return X

In [10]:
metatags["closest"] = ""
metatags["closest"] = metatags["summary"].apply(lambda x: getClosest(x))
metatags

,src,keywords,themes,summary,origin,url,closest
1,ea74b217d36ec570bb432fb5aa679090,"[AI panic, classrooms, education system, adapt...","[Impact of AI in Education, Adapting to AI in ...",The introduction of AI in classrooms has spark...,20230416,b'https://www.oneusefulthing.org/p/the-future-...,"[ea74b217d36ec570bb432fb5aa679090, adf886a1b9f..."
3,841546c0efc4c82f0aabc545a47a09e1,"[backdoor, cybersecurity researchers, encrypte...","[cybersecurity, encryption, vulnerabilities]",A group of cybersecurity researchers has disco...,20230810,b'https://www.vice.com/en/article/4a3n3j/backd...,"[841546c0efc4c82f0aabc545a47a09e1, f0da3ab471b..."
4,f6637d1fa0e3fe0e1e94bea8b3338ef9,"[Cybersecurity Researchers, Sandbox Satellite,...","[Satellite security, Hack-a-Sat competition, C...",Researchers have launched a satellite called M...,20230708,b'https://spaceref.com/newspace-and-tech/cyber...,"[f6637d1fa0e3fe0e1e94bea8b3338ef9, f21752e6152..."
5,58612e45effab1e47df9a86b14dfae85,"[regulating artificial intelligence, safeguard...","[Regulating AI, Ethical concerns, Technologica...",The text discusses the challenges of regulatin...,20231209,b'https://thegeneralist.substack.com/p/life-in...,"[58612e45effab1e47df9a86b14dfae85, c3301a7146d..."
6,7ff6239e13a43e856fa36887902c5ede,"[Chat-GPT, Python, Knowledge Graph, Neo4j, art...","[Artificial Intelligence, Natural Language Pro...",This article discusses how to use Chat-GPT and...,20290911,b'https://towardsdatascience.com/how-to-use-ch...,"[7ff6239e13a43e856fa36887902c5ede, af12c099700..."
...,...,...,...,...,...,...,...
1401,edbb7336dfb26098bcc966ca17c074b9,"[hanging out, book, Sheila Liming, crisis, iso...","[socializing, isolation, technology]","The text revolves around the concept of ""hangi...",20230305,b'https://slate.com/culture/2023/02/hanging-ou...,"[edbb7336dfb26098bcc966ca17c074b9, 7b316ebe449..."
1402,3fbb11d6e949d1e662aa6a146bb6cda0,"[insomniacs, sleeping assistants, Taobao, Chin...","[Insomnia, Sleeping services, Sleep economy]","China has a growing number of insomniacs, lead...",20221031,b'https://www.sixthtone.com/news/1011417?utm_s...,"[3fbb11d6e949d1e662aa6a146bb6cda0, 648de774fbf..."
1405,b9d91aca816a4b049d4583a774f886a2,"[Amazonian dark earth, soil, rainforest, ancie...","[Archaeology, Agriculture, Climate Change]",This article explores the importance of Amazon...,20240128,b'https://www.bbc.com/future/article/20240116-...,"[b9d91aca816a4b049d4583a774f886a2, 0c58d382a0c..."
1406,b79a2baa87b68283198416791b93bce4,"[U.S. government, Nvidia, high-performance com...","[Export regulations, China-US relations, AI de...",The U.S. government has imposed restrictions o...,20230320,b'https://www.tomshardware.com/news/us-bans-sa...,"[b79a2baa87b68283198416791b93bce4, 3c87907a359..."


In [11]:
metatags.to_parquet("data/pages_metatags_with_closest.parquet.gzip", compression="gzip")

# Closest done

In [12]:
metatags = pd.read_parquet("data/pages_metatags_with_closest.parquet.gzip")
metatags["kwsl"] = metatags["keywords"].apply(lambda lst: [x.lower().replace('"','') for x in lst])
metatags = metatags.merge(df[["src","title"]],on="src",how="left")
metatags

,src,keywords,themes,summary,origin,url,closest,kwsl,title
0,ea74b217d36ec570bb432fb5aa679090,"[AI panic, classrooms, education system, adapt...","[Impact of AI in Education, Adapting to AI in ...",The introduction of AI in classrooms has spark...,20230416,b'https://www.oneusefulthing.org/p/the-future-...,"[ea74b217d36ec570bb432fb5aa679090, adf886a1b9f...","[ai panic, classrooms, education system, adapt...",The Impact of AI on Education
1,841546c0efc4c82f0aabc545a47a09e1,"[backdoor, cybersecurity researchers, encrypte...","[cybersecurity, encryption, vulnerabilities]",A group of cybersecurity researchers has disco...,20230810,b'https://www.vice.com/en/article/4a3n3j/backd...,"[841546c0efc4c82f0aabc545a47a09e1, f0da3ab471b...","[backdoor, cybersecurity researchers, encrypte...",Researchers Uncover Backdoor in Encrypted Radi...
2,f6637d1fa0e3fe0e1e94bea8b3338ef9,"[Cybersecurity Researchers, Sandbox Satellite,...","[Satellite security, Hack-a-Sat competition, C...",Researchers have launched a satellite called M...,20230708,b'https://spaceref.com/newspace-and-tech/cyber...,"[f6637d1fa0e3fe0e1e94bea8b3338ef9, f21752e6152...","[cybersecurity researchers, sandbox satellite,...",Hack-a-Sat: Researchers Launch Sandbox Satelli...
3,58612e45effab1e47df9a86b14dfae85,"[regulating artificial intelligence, safeguard...","[Regulating AI, Ethical concerns, Technologica...",The text discusses the challenges of regulatin...,20231209,b'https://thegeneralist.substack.com/p/life-in...,"[58612e45effab1e47df9a86b14dfae85, c3301a7146d...","[regulating artificial intelligence, safeguard...",The Challenges of Regulating Artificial Intell...
4,7ff6239e13a43e856fa36887902c5ede,"[Chat-GPT, Python, Knowledge Graph, Neo4j, art...","[Artificial Intelligence, Natural Language Pro...",This article discusses how to use Chat-GPT and...,20290911,b'https://towardsdatascience.com/how-to-use-ch...,"[7ff6239e13a43e856fa36887902c5ede, af12c099700...","[chat-gpt, python, knowledge graph, neo4j, art...",Building a Knowledge Graph with Chat-GPT and P...
...,...,...,...,...,...,...,...,...,...
1110,edbb7336dfb26098bcc966ca17c074b9,"[hanging out, book, Sheila Liming, crisis, iso...","[socializing, isolation, technology]","The text revolves around the concept of ""hangi...",20230305,b'https://slate.com/culture/2023/02/hanging-ou...,"[edbb7336dfb26098bcc966ca17c074b9, 7b316ebe449...","[hanging out, book, sheila liming, crisis, iso...",The Radical Power of Hanging Out
1111,3fbb11d6e949d1e662aa6a146bb6cda0,"[insomniacs, sleeping assistants, Taobao, Chin...","[Insomnia, Sleeping services, Sleep economy]","China has a growing number of insomniacs, lead...",20221031,b'https://www.sixthtone.com/news/1011417?utm_s...,"[3fbb11d6e949d1e662aa6a146bb6cda0, 648de774fbf...","[insomniacs, sleeping assistants, taobao, chin...",China's Insomniacs Turn to Sleeping Assistants
1112,b9d91aca816a4b049d4583a774f886a2,"[Amazonian dark earth, soil, rainforest, ancie...","[Archaeology, Agriculture, Climate Change]",This article explores the importance of Amazon...,20240128,b'https://www.bbc.com/future/article/20240116-...,"[b9d91aca816a4b049d4583a774f886a2, 0c58d382a0c...","[amazonian dark earth, soil, rainforest, ancie...",The Ancient Agricultural Secret of Amazonian D...
1113,b79a2baa87b68283198416791b93bce4,"[U.S. government, Nvidia, high-performance com...","[Export regulations, China-US relations, AI de...",The U.S. government has imposed restrictions o...,20230320,b'https://www.tomshardware.com/news/us-bans-sa...,"[b79a2baa87b68283198416791b93bce4, 3c87907a359...","[u.s. government, nvidia, high-performance com...",US Restricts Nvidia GPU Sales to Thwart China'...


In [13]:
metatags[metatags.src == "feeb207dfea30efb1d5bf73503fd22a4"]["closest"].iloc[0]

array(['feeb207dfea30efb1d5bf73503fd22a4',
       'df59e2cf3380ffd9aeac9a3e01073300',
       'ed237776f4979a2104f62c4985fbeba8',
       'c42a95f16678ed3834840d48f8e775a3',
       '767b74c90576473294b2c47568c0e355',
       '5cf4407dc6fa3889e047c658e27c4ccf'], dtype=object)

In [14]:
for ix, row in metatags.iterrows():
    try:
        ID = row["src"]
        TXT = "# __"+row["title"] +"__, from (["+row.origin+"](https://kghosh.substack.com/p/"+row.origin+").)\n\n"
        TXT += "__[External link]("+str(row.url)[2:-1]+")__\n\n"
        TXT += "\n\n## Summary\n\n"
        TXT += row["summary"] + "\n\n## Keywords\n\n"
        TXT += "* "+"\n* ".join(row["keywords"])
        TXT += "\n\n## Themes\n\n"
        TXT += "* "+"\n* ".join(row["themes"])
        TXT += "\n\n## Signals\n\n"    
        SIG = signals[signals.src == ID]
        TXT += SIG[list(SIG.columns)[:-1]].to_markdown(index=False)
        if 1:
            TXT += "\n\n## Closest\n\n"
            TXT += "* "+"\n* ".join([ "["+df.loc[df["src"] == x,"title"].iloc[0]+"]("+x+")" for x in row["closest"][1:]])
        with open("docs/"+ID+".md", "w") as f:
            f.write(TXT) 
    except:
        print("Error with",row["src"])


# Now creating index

In [15]:
import random

In [16]:
allkw = []
for kw in metatags.themes:# keywords: #as an alternative
    allkw = allkw+list(kw)
KW = list(set(allkw))
KW = list(set([x.capitalize().replace('"','') for x in KW]))
KW = [x for x in KW if not x.isnumeric()]
KW.sort()
print(len(KW))
random.shuffle(KW)
KW[:10]

2763


['Packaging',
 'Employee layoffs',
 'Mvp definition',
 'Taxonomy and categorization',
 'Role of social media',
 'Rave culture',
 'History of debt',
 'Training with video games',
 'Generative ai',
 'Api development']

In [17]:
h = OAI.Helper("local_seeds_classif") 

In [18]:
with open("categories_small.csv","r") as f:
    ALL = f.read().split("\n")
ALL.sort()
ALL

['3D printing',
 'AI Applications',
 'AI Impacts',
 'AI Systems and Technology',
 'AI and Pop Culture ',
 'AI regulations and EU policies',
 'API development',
 'Access to culture',
 'Access to services',
 'Accounting',
 'Adoption and Use Cases of AI',
 'Adoption of generative AI',
 'Aerodynamics and engineering principles',
 'Affordability and accessibility of broadband services',
 'Aging and societal changes',
 'Agriculture and farming',
 'Airline Operations and Policies',
 'Algorithmic Model Optimization',
 'Animal behavior and cognition',
 'Animal communication',
 'Archaeological Discoveries and Research',
 'Architecture and social thought',
 'Art and Creativity',
 'Astronomy',
 'Attention Economy',
 'Automation',
 'Autonomous Systems and Agents',
 'Awards and recognition',
 'Branding and advertising',
 'Business and Economy',
 'Capitalism ',
 'Car subscriptions and mobility solutions',
 'Cartography',
 'Challenges ',
 'Chemistry and Composition',
 'Civic engagement and activism',


In [19]:
metatags

,src,keywords,themes,summary,origin,url,closest,kwsl,title
0,ea74b217d36ec570bb432fb5aa679090,"[AI panic, classrooms, education system, adapt...","[Impact of AI in Education, Adapting to AI in ...",The introduction of AI in classrooms has spark...,20230416,b'https://www.oneusefulthing.org/p/the-future-...,"[ea74b217d36ec570bb432fb5aa679090, adf886a1b9f...","[ai panic, classrooms, education system, adapt...",The Impact of AI on Education
1,841546c0efc4c82f0aabc545a47a09e1,"[backdoor, cybersecurity researchers, encrypte...","[cybersecurity, encryption, vulnerabilities]",A group of cybersecurity researchers has disco...,20230810,b'https://www.vice.com/en/article/4a3n3j/backd...,"[841546c0efc4c82f0aabc545a47a09e1, f0da3ab471b...","[backdoor, cybersecurity researchers, encrypte...",Researchers Uncover Backdoor in Encrypted Radi...
2,f6637d1fa0e3fe0e1e94bea8b3338ef9,"[Cybersecurity Researchers, Sandbox Satellite,...","[Satellite security, Hack-a-Sat competition, C...",Researchers have launched a satellite called M...,20230708,b'https://spaceref.com/newspace-and-tech/cyber...,"[f6637d1fa0e3fe0e1e94bea8b3338ef9, f21752e6152...","[cybersecurity researchers, sandbox satellite,...",Hack-a-Sat: Researchers Launch Sandbox Satelli...
3,58612e45effab1e47df9a86b14dfae85,"[regulating artificial intelligence, safeguard...","[Regulating AI, Ethical concerns, Technologica...",The text discusses the challenges of regulatin...,20231209,b'https://thegeneralist.substack.com/p/life-in...,"[58612e45effab1e47df9a86b14dfae85, c3301a7146d...","[regulating artificial intelligence, safeguard...",The Challenges of Regulating Artificial Intell...
4,7ff6239e13a43e856fa36887902c5ede,"[Chat-GPT, Python, Knowledge Graph, Neo4j, art...","[Artificial Intelligence, Natural Language Pro...",This article discusses how to use Chat-GPT and...,20290911,b'https://towardsdatascience.com/how-to-use-ch...,"[7ff6239e13a43e856fa36887902c5ede, af12c099700...","[chat-gpt, python, knowledge graph, neo4j, art...",Building a Knowledge Graph with Chat-GPT and P...
...,...,...,...,...,...,...,...,...,...
1110,edbb7336dfb26098bcc966ca17c074b9,"[hanging out, book, Sheila Liming, crisis, iso...","[socializing, isolation, technology]","The text revolves around the concept of ""hangi...",20230305,b'https://slate.com/culture/2023/02/hanging-ou...,"[edbb7336dfb26098bcc966ca17c074b9, 7b316ebe449...","[hanging out, book, sheila liming, crisis, iso...",The Radical Power of Hanging Out
1111,3fbb11d6e949d1e662aa6a146bb6cda0,"[insomniacs, sleeping assistants, Taobao, Chin...","[Insomnia, Sleeping services, Sleep economy]","China has a growing number of insomniacs, lead...",20221031,b'https://www.sixthtone.com/news/1011417?utm_s...,"[3fbb11d6e949d1e662aa6a146bb6cda0, 648de774fbf...","[insomniacs, sleeping assistants, taobao, chin...",China's Insomniacs Turn to Sleeping Assistants
1112,b9d91aca816a4b049d4583a774f886a2,"[Amazonian dark earth, soil, rainforest, ancie...","[Archaeology, Agriculture, Climate Change]",This article explores the importance of Amazon...,20240128,b'https://www.bbc.com/future/article/20240116-...,"[b9d91aca816a4b049d4583a774f886a2, 0c58d382a0c...","[amazonian dark earth, soil, rainforest, ancie...",The Ancient Agricultural Secret of Amazonian D...
1113,b79a2baa87b68283198416791b93bce4,"[U.S. government, Nvidia, high-performance com...","[Export regulations, China-US relations, AI de...",The U.S. government has imposed restrictions o...,20230320,b'https://www.tomshardware.com/news/us-bans-sa...,"[b79a2baa87b68283198416791b93bce4, 3c87907a359...","[u.s. government, nvidia, high-performance com...",US Restricts Nvidia GPU Sales to Thwart China'...


In [20]:
import json
with open("data/categories_ax.json","r") as f:
    ax = json.loads(f.read())
with open("data/categories_bx.json","r") as f:
    bx = json.loads(f.read())

In [22]:

for ix, row in metatags.iterrows():
    if ix < 100000: # useless
        ax[row["src"]] = {}
        if row["src"] not in ax.keys():
            PROMPT = "You are given a text below. You need to pick the five most relevant themes of this text from the list of keywords below:\n\n* "+"\n*".join(ALL)+"\n\n\nAnswer only with 3 items of this list, separated by commas"
            ANS =  h.ask(PROMPT,row["summary"],src="tag",v="gpt-3.5-turbo-0125")
            LST = [x.strip() for x in ANS.split(",") if x.strip() in ALL]
            ax[row["src"]] = LST
            for x in LST:
                if x not in bx.keys():
                    bx[x] = []
                bx[x].append(row["src"])

In [23]:
import json
with open("data/categories_ax.json","w") as f:
    f.write(json.dumps(ax))
with open("data/categories_bx.json","w") as f:
    f.write(json.dumps(bx))

In [24]:
import string
LETTERS = string.ascii_uppercase
LETTERS

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [25]:
bx['Epidemic Outbreaks and Response'][0]

'b1b5f0564e486b3e1f123da956f6b1a6'

In [26]:
INDEX  = "# Overview \n\n"
INDEX += "This page tries to capture the information obtained in my [tech watch](https://substack.kghosh.me/). The different links and pages are captured below, trying to catalogue different weak signals." 


for letter in LETTERS:
    kws = [x for x in list(bx.keys()) if x[0] == letter]

    if len(kws):
        INDEX += "\n\n## Letter '"+letter.upper()+"' \n\n"
        for kw in kws:
            INDEX += "\n### Category: __"+kw+"__: \n\n"
            lst = bx[kw] 
            for x in lst:
                TITRE = df.loc[df.src == x,"title"].iloc[0]
                INDEX += "* ["+TITRE+"]("+x+")\n"

INDEX += "\n\n## Others\n\n"
for x in ax:
    if len(ax[x]) == 0:
        TITRE = df.loc[df.src == x,"title"].iloc[0]
        INDEX += "* ["+TITRE+"]("+x+")\n"

In [27]:
with open("docs/index.md", "w") as f:
    f.write(INDEX) 